In [11]:
using JuMP, Gurobi, LinearAlgebra, Random, Statistics, MAT

In [2]:
J = 4
M = 2

TE = [18, 14, 12, 6]
TC = [4, 4, 6, 6]
st = [2, 4];

In [3]:
v1 = Model(Gurobi.Optimizer)

@variable(v1, x[0:J,0:J,1:M], Bin)#variavel de ordem
@variable(v1, E[0:J]≥0, Int)#variavel de tempo de enfesto
@variable(v1, C[0:J]≥0, Int)#variavel de tempo de enfesto
@variable(v1, time>=0)#variavel para funcao objetivo

@constraint(v1, E[0]==0)#enfesto 0
@constraint(v1 , C[0]==0)#corte 0
@constraint(v1 , [i=0:J,m=1:M], x[i,i,m]==0)
@constraint(v1, [i=1:J,m=1:M], sum(x[i,j,m] for j in 0:J)==1)
@constraint(v1, [j=1:J,m=1:M], sum(x[i,j,m] for i in 0:J)==1)
@constraint(v1, [m=1:2], sum(x[0,j,m] for j in 1:J)≤ 1)
@constraint(v1, [i=0:J,j=1:J], E[j]≥E[i]+st[1]+TE[j]-(1-x[i,j,1])*10000)
@constraint(v1, [i=0:J,j=1:J], C[j]≥C[i]+st[2]+TC[j]-(1-x[i,j,2])*10000)
@constraint(v1, [i=1:J], C[i]≥E[i]+st[2]+TC[i])
@constraint(v1, [i=1:J], time ≥ C[i]);

@objective(v1, Min, time)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 941647
Academic license 941647 - for non-commercial use only - registered to si___@cos.ufrj.br


time

In [4]:
optimize!(v1)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-9600K CPU @ 3.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Academic license 941647 - for non-commercial use only - registered to si___@cos.ufrj.br
Optimize a model with 78 rows, 61 columns and 220 nonzeros
Model fingerprint: 0x7ed22278
Variable types: 1 continuous, 60 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 76.0000000
Presolve removed 24 rows and 20 columns
Presolve time: 0.00s
Presolved: 54 rows, 41 columns, 160 nonzeros
Variable types: 0 continuous, 41 integer (32 binary)

Root relaxation: objective 1.000000e+01, 34 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

In [5]:
objective_value(v1)

66.0

In [6]:
value_x = zeros((5,5,2))
value_x[:,:,:] = value.(v1[:x])[:,:,:]

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 0:4
    Dimension 2, 0:4
    Dimension 3, Base.OneTo(2)
And data, a 5×5×2 Array{Float64, 3}:
[:, :, 1] =
 0.0  -0.0  -0.0  -0.0   1.0
 0.0   0.0   1.0  -0.0  -0.0
 1.0  -0.0   0.0  -0.0  -0.0
 0.0   1.0  -0.0   0.0  -0.0
 0.0  -0.0  -0.0   1.0   0.0

[:, :, 2] =
 0.0  -0.0  -0.0   1.0  -0.0
 0.0   0.0   1.0  -0.0  -0.0
 1.0  -0.0   0.0  -0.0  -0.0
 0.0  -0.0  -0.0   0.0   1.0
 0.0   1.0  -0.0  -0.0   0.0

In [7]:
value_E = zeros(5)
value_E[:] = value.(v1[:E])[:]

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:4
And data, a 5-element Vector{Float64}:
  0.0
 42.0
 58.0
 22.0
  8.0

In [8]:
value_C = zeros(5)
value_C[:] = value.(v1[:C])[:]

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:4
And data, a 5-element Vector{Float64}:
  0.0
 50.0
 66.0
 32.0
 42.0

In [9]:
Dicionario = Dict(
"x" => value_x,
"C" => value_C,
"E" => value_E        
)

Dict{String, Array{Float64}} with 3 entries:
  "C" => [0.0, 50.0, 66.0, 32.0, 42.0]
  "x" => [0.0 -0.0 … -0.0 1.0; 0.0 0.0 … -0.0 -0.0; … ; 0.0 1.0 … 0.0 -0.0; 0.0…
  "E" => [0.0, 42.0, 58.0, 22.0, 8.0]

In [12]:
matwrite("v1.mat", Dicionario)